# Aspect-Based Sentiment Analysis for Restaurant Review


---


Kelompok 8 nlp.zip :

Hengky Surya Angkasa - 13518048

Gunawan Kamaruddin - 13518103

Chokyi Ozer - 13518107



#Overview


---
Aplikasi ini memiliki fungsionalitas utama untuk mengklasifikasikan sebuah review (*in-domain restaurant*) ke dalam sentimen **positif** atau **negatif**.

Satu review dapat memberikan hasil klasifikasi beberapa sentimen bergantung pada aspek-aspek yang terdapat di dalamnya. Sistem akan meng-ekstrak aspek-aspek review yang terdapat dalam kalimat dan kemudian memberikan mengklasifikan masing-masing aspek tersebut ke beberapa kategori.

Sistem kemudian akan menghasilkan sentimen berdasarkan hasil ekstraksi dan klasifikasi aspek.

Secara umum alur dari sistem ini adalah sebagai berikut.



1.   Ekstraksi aspek pada review -> Menghasilkan satu atau lebih *aspect terms*
2.   Klasifikasi *aspect terms* yang dihasilkan -> Menghasilkan satu atau lebih *aspect category*
3.   Klasifikasi sentimen review berdasarkan *aspect category* -> Menghasilkan satu atau lebih nilai sentimen yang terikat dengan *aspect term* dan *aspect category*



# 1. Demo Preparation
Persiapan mencakup meng-impor kakas-kakas yang diperlukan dan mengunduh file-file yang berhubungan dengan sistem, termasuk model yang akan digunakan.

## 1.1 File Loading

In [1]:
from zipfile import *

# Training and Test Data
!wget -P /root/input/ -c "https://raw.githubusercontent.com/TFGungun/ABSA-NLP2021Data/master/dataset/train.csv"
!wget -P /root/input/ -c "https://raw.githubusercontent.com/TFGungun/ABSA-NLP2021Data/master/dataset/test.csv"
!wget -P /root/input/ -c "https://raw.githubusercontent.com/TFGungun/ABSA-NLP2021Data/master/dataset/test_gold.csv"

# Aspect Extraction Model
!wget -O /root/input/ae_bert_linear.h5 "https://github.com/TFGungun/ABSA-NLP2021Data/blob/master/AspectExtraction/ae_bert_linear.h5?raw=true"
!wget -O /root/input/ae_crf.crfsuite "https://github.com/TFGungun/ABSA-NLP2021Data/blob/master/AspectExtraction/ae_crf.crfsuite?raw=true"

# Aspect Classification Model
!wget -O /root/input/TfidfVectorizer.pickle "https://github.com/TFGungun/ABSA-NLP2021Data/blob/master/AspectClassification/TfidfVectorizer.pickle?raw=true"
!wget -O /root/input/AspectClassificationModel.pickle "https://github.com/TFGungun/ABSA-NLP2021Data/blob/master/AspectClassification/AspectClassificationModel.pickle?raw=true"
!wget -O /root/input/AspectClassificationModel_New.pickle "https://github.com/TFGungun/ABSA-NLP2021Data/blob/master/AspectClassification/AspectClassificationModel_New.pickle?raw=true"

# Sentiment Analysis Model
!wget -O /root/input/KerasTokenizer.pickle "https://github.com/TFGungun/ABSA-NLP2021Data/blob/master/SentimentAnalysis/KerasTokenizer.pickle?raw=true"
!wget -O /root/input/AspectClassificationModel.pickle "https://github.com/TFGungun/ABSA-NLP2021Data/blob/master/AspectClassification/AspectClassificationModel.pickle?raw=true"
!wget -O /root/input/SentimentAnalysisModel.zip "https://github.com/TFGungun/ABSA-NLP2021Data/blob/master/SentimentAnalysis/SentimentAnalysisModel.zip?raw=true"
SA_MODEL_ZIP_FILE = '/root/input/SentimentAnalysisModel.zip'
with ZipFile(SA_MODEL_ZIP_FILE, 'r') as zipObj:
   zipObj.extractall('/root/input/')

# Google Word2Vec Corpus
#!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-11-16 09:15:08--  https://raw.githubusercontent.com/TFGungun/ABSA-NLP2021Data/master/dataset/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 310367 (303K) [text/plain]
Saving to: ‘/root/input/train.csv’

train.csv           100%[===================>] 303.09K  --.-KB/s    in 0.03s   

2021-11-16 09:15:08 (8.95 MB/s) - ‘/root/input/train.csv’ saved [310367/310367]

--2021-11-16 09:15:08--  https://raw.githubusercontent.com/TFGungun/ABSA-NLP2021Data/master/dataset/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Leng

## 1.2 Libraries Import

In [2]:
!pip install -q transformers
!pip install -q tf-models-official

     |████████████████████████████████| 3.1 MB 5.0 MB/s 
     |████████████████████████████████| 596 kB 35.6 MB/s 
     |████████████████████████████████| 895 kB 72.6 MB/s 
     |████████████████████████████████| 3.3 MB 47.1 MB/s 
     |████████████████████████████████| 59 kB 7.0 MB/s 
     |████████████████████████████████| 1.8 MB 5.2 MB/s 
     |████████████████████████████████| 213 kB 56.4 MB/s 
     |████████████████████████████████| 4.9 MB 40.2 MB/s 
     |████████████████████████████████| 352 kB 44.5 MB/s 
     |████████████████████████████████| 90 kB 9.5 MB/s 
     |████████████████████████████████| 1.2 MB 34.4 MB/s 
     |████████████████████████████████| 47.6 MB 36 kB/s 
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 99 kB 7.8 MB/s 
     |████████████████████████████████| 1.1 MB 44.7 MB/s 


In [3]:
import numpy as np
import re
import pandas as pd
import pickle

# Aspect Extraction
import numpy as np
import spacy

import tensorflow as tf
from official.nlp import optimization
from transformers import BertTokenizer, TFBertMainLayer

# Aspect Classification
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import TextVectorization, Dense, Embedding, Input, Flatten, Dropout

# Sentiment Analysis
import gensim
from gensim.models import KeyedVectors
from sklearn import model_selection
import keras
from keras import layers, models
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


# 2. Term Extraction
Persiapan model ekstraksi aspek untuk kalimat review

In [4]:
class AspectTermExtractor():
    def tokenize(self, sentence: str):
        nlp = spacy.load("en_core_web_sm")
        doc = nlp(sentence)
        return [token.text for token in doc]
    
    def extract_aspect(self, sentence: str):
        pass

In [5]:
class BERTATE(AspectTermExtractor):
    tag2idx = {'B': 2, 'I': 0, 'O': 1, 'PAD': 3}

    def __init__(self) -> None:
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

        # Generate optimizer
        epochs = 5
        num_train_steps = 7680
        num_warmup_steps = int(0.1*num_train_steps)

        init_lr = 3e-5
        optimizer = optimization.create_optimizer(init_lr=init_lr,
                                                num_train_steps=num_train_steps,
                                                num_warmup_steps=num_warmup_steps,
                                                optimizer_type='adamw')

        loss = loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
        metrics = ['accuracy']

        # bert_model = TFBertModel.from_pretrained(
        #     "bert-base-cased",
        #     num_labels=len(self.tag2idx),
        #     output_attentions = False,
        #     output_hidden_states = False
        # )

        model_file = "/root/input/ae_bert_linear.h5"
        self.model = keras.models.load_model(model_file, custom_objects={"bert": TFBertMainLayer})

        self.model.compile(optimizer=optimizer,
                    loss=loss,
                    metrics=metrics)

    def tokenize(self, sentence: str):
        word_tokens = super().tokenize(sentence)
        bert_tokens = []
        for token in word_tokens:
            bert_tokens.extend(self.tokenizer.tokenize(token))
        
        return bert_tokens

    def extract_aspect(self, sentence: str):
        from keras.preprocessing.sequence import pad_sequences

        tokens = self.tokenize(sentence)
        padded = pad_sequences([self.tokenizer.convert_tokens_to_ids(tokens)],
                              maxlen=64, dtype="long", value=0.0,
                              truncating="post", padding="post")[0]

        masks = [float(i != 0.0) for i in padded]

        padded = np.array([padded])
        masks = np.array([masks])

        pred = self.model.predict([padded, masks])
        pred_tags = np.argmax(pred[0], axis=1)

        # Init state
        aspects = []
        is_aspect = False
        aspect_idx = 0
        last_idx = 0
        for i in range(len(tokens)):
            if i >= 64:
                break
            if tokens[i].startswith("##"):
                idx = sentence.find(tokens[i][2:], last_idx)
            else:
                idx = sentence.find(tokens[i], last_idx)

            if is_aspect:
                if pred_tags[i] == self.tag2idx['B']:
                    # Skip if is ##
                    if not tokens[i].startswith("##"):
                        aspects.append(sentence[aspect_idx:idx].strip())
                        aspect_idx = idx
                elif pred_tags[i] == self.tag2idx['I']:
                    pass
                else: #pred_tags[i] == self.tag2idx['O']
                    # Return to not-aspect if it is not a sub word
                    if not tokens[i].startswith("##"):
                        aspects.append(sentence[aspect_idx:idx].strip())
                        is_aspect = False
            else:
                if pred_tags[i] == self.tag2idx['B']:
                    if not tokens[i].startswith("##"):
                        aspect_idx = idx
                        is_aspect = True
                elif pred_tags[i] == self.tag2idx['I']:
                    if not tokens[i].startswith("##"):
                        aspect_idx = idx
                        is_aspect = True
                else: # pred_tags[i] == self.tag2idx['O']
                    pass
            
            last_idx = idx + 1
        
        if is_aspect:
            aspects.append(sentence[aspect_idx:].strip())
        
        return aspects

In [6]:
extractor = BERTATE()

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

INFO:absl:using Adamw optimizer
INFO:absl:gradient_clip_norm=1.000000


# 3. Category Classification
Persiapan model klasifikasi aspek untuk *aspect terms* yang berasal dari pipeline sebelumnya

In [7]:
class AspectClassificationModel:
  def __init__(self):
    self.model = pickle.load(open('/root/input/AspectClassificationModel_New.pickle', 'rb'))
    df_train = pd.read_csv("/root/input/train.csv")
    df_train = df_train.dropna()
    y = df_train["aspect"]

    self.label_encoder = LabelEncoder()
    category_encoded = tf.keras.utils.to_categorical(self.label_encoder.fit_transform(y))

    vocab_size = 10000
    sequence_length = 100
    embedding_dim = 32

    vectorize_layer = TextVectorization(
        max_tokens=vocab_size,
        output_mode='int',
        output_sequence_length=sequence_length)

    df_term = df_train.loc[:,'term']
    term = df_term.values
    vectorize_layer.adapt(term)

    self.vectorize_model = Sequential()
    self.vectorize_model.add(Input(shape=(1,), dtype=tf.string))
    self.vectorize_model.add(vectorize_layer)

  def predict(self, terms):
    y_pred = self.model.predict(self.vectorize_model.predict(terms))
    y_pred = self.label_encoder.inverse_transform(np.argmax(y_pred,axis=1))
    return y_pred

In [8]:
ac_model = AspectClassificationModel()

# 4. Aspect Based Sentiment Analysis
Persiapan model klasifikasi sentimen untuk *aspect category* yang berasal dari pipeline sebelumnya


In [21]:
class SentimentAnalysisDataHelper:
  SENTENCE_MAX_LEN = 61
  def __init__(self):
      self.tokenizer = pickle.load(open('/root/input/KerasTokenizer.pickle', 'rb'))

  def clean_str(self, string):
      """
      Tokenization/string cleaning for all datasets except for SST.
      Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
      """
      string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
      string = re.sub(r"\'s", " \'s", string)
      string = re.sub(r"\'ve", " \'ve", string)
      string = re.sub(r"n\'t", " n\'t", string)
      string = re.sub(r"\'re", " \'re", string)
      string = re.sub(r"\'d", " \'d", string)
      string = re.sub(r"\'ll", " \'ll", string)
      string = re.sub(r",", " , ", string)
      string = re.sub(r"!", " ! ", string)
      string = re.sub(r"\(", " \( ", string)
      string = re.sub(r"\)", " \) ", string)
      string = re.sub(r"\?", " \? ", string)
      string = re.sub(r"\s{2,}", " ", string)
      return string.strip().lower()


  def load_data_and_labels(self):
      # Load data from file
      dtf = pd.read_csv('/root/input/train.csv', index_col = 0)[['review', 'aspect', 'sentiment']]
      # Split by words
      x_text = dtf['review']
      x_text = [clean_str(sent) for sent in x_text]
      # Generate labels
      y_labels = dtf['sentiment']
      y = [[1, 0] if y == "positive" else [0, 1] for y in y_labels]
      # Aspect category
      aspect_parent = [aspect.split('#')[0] for aspect in dtf['aspect']]
      aspect_child = [aspect.split('#')[1] for aspect in dtf['aspect']]
      for i in range(0, len(aspect_child)):
          if(aspect_child[i] == "STYLE_OPTIONS"):
              aspect_child[i] = "STYLE"
      return [x_text, y, aspect_parent, aspect_child]
      
  def process_review_and_aspect(self, review, aspect):
      data_sequence = self.tokenizer.texts_to_sequences([self.clean_str(review)])
      aspect_sequence = self.tokenizer.texts_to_sequences([self.clean_str(aspect)])
      for i in range(0, len(data_sequence)):
        aspect_sequence[i] = aspect_sequence[i] * np.count_nonzero(data_sequence[i])

      data_sequence = pad_sequences(data_sequence, maxlen=self.SENTENCE_MAX_LEN, padding="post", truncating="post")
      aspect_sequence = pad_sequences(aspect_sequence, maxlen=self.SENTENCE_MAX_LEN, padding="post", truncating="post")


      return data_sequence, aspect_sequence

In [22]:
class SentimentAnalysisModel:
  def __init__(self):
    self.model = keras.models.load_model('/root/input/CNNAspectParentModel')
    self.data_helper = SentimentAnalysisDataHelper()

  def process_output(self, output):
    if(output[0] > output[1]):
      return "Positive"
    else:
      return "Negative"

  def predict(self, review, aspect):
    input = [self.data_helper.process_review_and_aspect(review, aspect)]
    output = self.model.predict(input)
    output_processed = self.process_output(output[0])
    return output_processed
  
  def predict_multiple_aspect(self, review, aspectArr):
    input_review = []
    input_aspect = []
    for aspect in aspectArr:
      review_processed , aspect_processed = self.data_helper.process_review_and_aspect(review, aspect)
      input_review.extend(review_processed)
      input_aspect.extend(aspect_processed)
    input_review = np.asarray(input_review)
    input_aspect = np.asarray(input_aspect)
    output_raw = self.model.predict([input_review, input_aspect])

    output_processed = []
    for output in output_raw:
      output_processed.extend([self.process_output(output)])
    
    return output_processed


In [23]:
sa_model = SentimentAnalysisModel()


# 5. End-to-End Prediction
Program penuh yang dapat digunakan untuk mengklasifikasikan review.

Program menggunakan input terminal sebagai masukan utama. Untuk keluar dari program digunakan perintah "exit".

In [24]:
review = input("Enter review here : ")

while review != "exit":
  review_terms = extractor.extract_aspect(review)
  review_category = ac_model.predict(review_terms)
  review_category_parent = [aspect.split('#')[0] for aspect in review_category]
  review_sentiment = sa_model.predict_multiple_aspect(review, review_category)
  print("Review : " + review)
  print("Sentiment : ")
  for i in range(0, len(review_sentiment)):
    print("\t" + review_sentiment[i] + " opinion about " + review_terms[i] + " (" + review_category[i] + ")")
  review = input("Enter review here : ")

Enter review here :  Nice ambience, but highly overrated place
Review :  Nice ambience, but highly overrated place
Sentiment : 
	Positive opinion about ambience (AMBIENCE#GENERAL)
	Positive opinion about place (RESTAURANT#GENERAL)
Enter review here : I don't wanna come back here anymore, the place is awful, the food is bad
Review : I don't wanna come back here anymore, the place is awful, the food is bad
Sentiment : 
	Positive opinion about place (RESTAURANT#GENERAL)
	Positive opinion about food (FOOD#QUALITY)
Enter review here : the food was great, the margaritas too but the waitress was too busy being nice to her other larger party than to take better care of my friend and me.
Review : the food was great, the margaritas too but the waitress was too busy being nice to her other larger party than to take better care of my friend and me.
Sentiment : 
	Positive opinion about food (FOOD#QUALITY)
	Positive opinion about margaritas (DRINKS#QUALITY)
	Negative opinion about waitress (SERVICE#